In [ ]:
# 크롤링에 필요한 모듈 가져오기
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from time import sleep
import random
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def scroll(modal,driver):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

In [ ]:
df_dict = dict()
def page_scoll(url,name) :
    star_list = []
    sympathy_list = []
    comment_list = []
        # 크롬 드라이버 세팅
    driver = webdriver.Chrome('chromedriver.exe')
        # 페이지 열기
    driver.get(url)
    # 페이지 로딩 대기
    wait = WebDriverWait(driver, 3)

    # '리뷰 모두 보기' 버튼 렌더링 확인(path 수정 @2022-06-22)
    all_review_button_xpath = '/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
    button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))
    # '리뷰 모두 보기' 버튼 클릭
    driver.find_element(By.XPATH,all_review_button_xpath).click()

    # '리뷰 모두 보기' 페이지 렌더링 대기
    all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
    page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

    # 페이지 무한 스크롤 다운
    modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
    scroll(modal,driver)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_source = soup.select('div.RHo1pe')
    for i in range(0,len(review_source)):
        comment_list.append(soup.select("div.RHo1pe > div.h3YV2d")[i].text)
        star = driver.find_element(By.XPATH,f'//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div/div[1]/div[{i+1}]/header/div[2]/div')
        star = star.get_attribute("aria-label")[10]
        star_list.append(star)
        try:
            sympathy = driver.find_element(By.CSS_SELECTOR,f'#yDmH0d > div.VfPpkd-Sx9Kwc.cC1eCc.UDxLd.PzCPDd.HQdjr.VfPpkd-Sx9Kwc-OWXEXe-FNFY6c > div.VfPpkd-wzTsW > div > div > div > div > div.fysCi > div > div:nth-child(1) > div:nth-child({i+1}) > div:nth-child(3) > div').text[4:7]
            sympathy = sympathy.replace('명','')
            sympathy = sympathy.replace('이','')
            sympathy_list.append(sympathy)
        except:
            sympathy_list.append(0)
    
    df1 = pd.DataFrame(star_list)
    df2 = pd.DataFrame(sympathy_list)
    df3 = pd.DataFrame(comment_list)
    df = pd.concat([df1,df2,df3],axis=1)
    df.columns = ["별점","공감","댓글"]
            
    # df = pd.DataFrame([i for i in comment_list])
    df_dict[name]=df
    
    return df_dict

In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.petsbe.android.petsbemall','어바웃펫')    # 함수를 통해 앱사이트 댓글 크롤링
df = df_dict['어바웃펫']                                                                      # 크롤링 댓글 --> DataFrame 형식 적용
df.to_csv('./어바웃펫.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.fineappstudio.android.petfriends','펫프렌즈')    # 함수를 통해 앱사이트 댓글 크롤링
df = df_dict['펫프렌즈']                                                                      # 크롤링 댓글 --> DataFrame 형식 적용
df.to_csv('./펫프렌즈.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=kr.co.fitpet','핏펫')    # 함수를 통해 앱사이트 댓글 크롤링
df = df_dict['핏펫']                                                                      # 크롤링 댓글 --> DataFrame 형식 적용
df.to_csv('./핏펫.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.applepie4.mylittlepet','헬로펫')
df = df_dict['헬로펫']
df.to_csv('./헬로펫.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.samsungcard.pet','아지냥이')
df = df_dict['아지냥이']
df.to_csv('./아지냥이.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.banlife.banlifeapp','반려생활')
df = df_dict['반려생활']
df.to_csv('./반려생활.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.radcns.petpot','펫팟')
df = df_dict['펫팟']
df.to_csv('./펫팟.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.petbom','펫봄')
df = df_dict['펫봄']
df.to_csv('./펫봄.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.funnc.dogpre','강아지대통령')
df = df_dict['강아지대통령']
df.to_csv('./강아지대통령.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.saladpet.app','샐러드펫')
df = df_dict['샐러드펫']
df.to_csv('./샐러드펫.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=kr.bemypet.bemypet_main_app','비마이펫')
df = df_dict['비마이펫']
df.to_csv('./비마이펫.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.ylcompany.ahapet','아하펫')
df = df_dict['아하펫']
df.to_csv('./아하펫.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.petbox.shop','펫박스')
df = df_dict['펫박스']
df.to_csv('./펫박스.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=io.efil.moco','멍냥보감')
df = df_dict['멍냥보감']
df.to_csv('./멍냥보감.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.butlerdiary.butlerdiary','집사일기')
df = df_dict['집사일기']
df.to_csv('./집사일기.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=kr.co.petfins.android','펫핀스')
df = df_dict['펫핀스']
df.to_csv('./펫핀스.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.dfang.playfang','펫피')
df = df_dict['펫피']
df.to_csv('./펫피.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.piedpuppy','피리부는 강아지')
df = df_dict['피리부는 강아지']
df.to_csv('./피리부는 강아지.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.potpleapps.holapet','올라펫')
df = df_dict['올라펫']
df.to_csv('./올라펫.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=kr.co.vango.android','반려의고수')
df = df_dict['반려의고수']
df.to_csv('./반려의고수.csv', index=False)

스크롤 완료


In [ ]:
page_scoll('https://play.google.com/store/apps/details?id=com.petudy.mobile','펫터디')
df = df_dict['펫터디']
df.to_csv('./펫터디.csv', index=False)